# 操作步示例

## 第一步：使用Qwen检索相关文献

In [ ]:
import openai
import requests
import yaml
import time

# ------------------------------------------------------------
# 设置 Qwen API 的认证信息
# ------------------------------------------------------------
# 请确保 API 密钥安全，生产环境中建议通过配置或环境变量传递敏感信息
openai.api_key = "sk-08ae8b6ba977447cba853e9b617fa10d"
openai.api_base = "https://dashscope.aliyuncs.com/compatible-mode/v1"

# ------------------------------------------------------------
# 函数：fetch_crossref
# 作用：调用 CrossRef API 根据查询条件获取文献数据
# 参数：
#   query      - 查询字符串（例如方向和关键词的组合）
#   year_range - 出版年份范围，格式如 "2018-2025"
# 返回值：一个包含文献信息的列表
# ------------------------------------------------------------
def fetch_crossref(query, year_range):
    url = "https://api.crossref.org/works"
    params = {
        "query": query,
        "filter": f"from-pub-date:{year_range.split('-')[0]},until-pub-date:{year_range.split('-')[1]}",
        "rows": 50  # 获取50篇文献，可根据实际需要修改
    }
    headers = {
        "User-Agent": "DocumentManager/1.0 (mailto:your_email@example.com)"  # 建议替换为你自己的邮箱
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        return response.json().get('message', {}).get('items', [])
    else:
        print(f"CrossRef请求失败：{response.status_code}")
        return []

# ------------------------------------------------------------
# 函数：read_config
# 作用：从 config.yaml 文件中读取配置参数（方向、关键词、出版年份范围）
# 返回值：一个字典，包含配置信息
# ------------------------------------------------------------
def read_config():
    with open("config.yaml", "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

# ------------------------------------------------------------
# 函数：qwen_api_call
# 作用：调用 Qwen API 生成对话回复（这里用于返回文献相关性评分）
# 参数：
#   prompt - 发送到模型的提示文本
#   model  - 使用的模型名称，默认使用 "qwen-max"
# 返回值：API 返回的文本（这里应为一个评分的数字字符串）
# ------------------------------------------------------------
def qwen_api_call(prompt, model="qwen-max"):
    try:
        completion = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,  # 保证输出稳定
            top_p=1.0,
            max_tokens=10    # 输出长度，根据需要可调大
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print("Qwen API调用失败：", e)
        return ""

# ------------------------------------------------------------
# 函数：ai_filter_qwen
# 作用：利用 Qwen API 对文献与设定方向的相关性进行评分，并判断是否超过阈值
# 参数：
#   item      - 单篇文献预处理后的数据（包含标题、摘要等）
#   direction - 给定的研究方向或主题
#   threshold - 评分阈值（默认0.6，>=这个值则认为相关）
# 返回值：如果文献相关返回 True，否则返回 False
# ------------------------------------------------------------
def ai_filter_qwen(item, direction, threshold=0.6):
    prompt = f"""
请判断以下文献是否与“{direction}”密切相关，并请仅返回一个0到1之间的评分（保留两位小数）。

标题：{item.get('title', '无标题')}
摘要：{item.get('abstract', '无摘要')}

评分标准：
0.00 表示完全不相关，1.00 表示高度相关。
"""
    try:
        response = qwen_api_call(prompt)
        score_str = response.strip()
        score = float(score_str)
        print(f"文献《{item['title'][:30]}...》评分：{score}")
        return score >= threshold
    except Exception as e:
        print(f"评分解析错误：{e}")
        return False

# ------------------------------------------------------------
# 函数：save_filtered_items
# 作用：将过滤后的文献信息保存至指定文件（这里使用 YAML 格式）
# ------------------------------------------------------------
def save_filtered_items(filtered_items, filename="filtered_items.yaml"):
    with open(filename, "w", encoding="utf-8") as f:
        yaml.safe_dump(filtered_items, f, allow_unicode=True)
    print(f"过滤后的文献信息已保存至 {filename}")

# ------------------------------------------------------------
# 主函数
# ------------------------------------------------------------
def main():
    config = read_config()
    direction = config.get("direction", "")
    keywords = " ".join(config.get("keywords", []))
    publication_year = config.get("publication_year", "")
    
    query = f"{direction} {keywords}"
    items = fetch_crossref(query, publication_year)
    
    # 预处理文献数据，提取关键字段
    processed_items = []
    for item in items:
        processed = {
            "title": item.get("title", [""])[0] if item.get("title") else "无标题",
            "abstract": item.get("abstract", "无摘要"),
            "authors": item.get("author", []),
            "date": item.get("issued", {}).get("date-parts", [[""]])[0][0],
            "DOI": item.get("DOI", "")
        }
        processed_items.append(processed)
    
    print(f"初步获取到 {len(processed_items)} 篇文献。")
    
    # 使用 Qwen API 对文献进行过滤，根据相关性评分判断是否通过
    filtered_items = []
    for item in processed_items:
        if ai_filter_qwen(item, direction):
            filtered_items.append(item)
            print(f"文献《{item['title']}》通过Qwen过滤。")
        else:
            print(f"文献《{item['title']}》未通过Qwen过滤。")
        time.sleep(1)  # 暂停1秒，防止请求过于频繁
    
    print(f"经过Qwen过滤后，剩余文献数量：{len(filtered_items)}")
    
    # 保存过滤后的文献信息到文件
    save_filtered_items(filtered_items, filename="filtered_items.yaml")

# 当脚本作为主程序执行时，调用 main() 函数
if __name__ == "__main__":
    main()


初步获取到 50 篇文献。
文献《基于在生物教学中激发学生学习兴趣的研究...》评分：0.0
文献《基于在生物教学中激发学生学习兴趣的研究》未通过Qwen过滤。
文献《基于深度学习的工程图像识别与分类研究...》评分：0.05
文献《基于深度学习的工程图像识别与分类研究》未通过Qwen过滤。
文献《CT-CPI: Deep Learning Model of...》评分：0.05
文献《CT-CPI: Deep Learning Model of Compound-Protein Interaction Based on Integrated CNN Module and Transformer》未通过Qwen过滤。
文献《基于深度学习的工程检测智能化创新研究...》评分：0.01
文献《基于深度学习的工程检测智能化创新研究》未通过Qwen过滤。
文献《Research Progress of Predictin...》评分：0.05
文献《Research Progress of Predicting Long Non-Coding RNA-MicroRNA Interaction Based on Deep Learning》未通过Qwen过滤。
文献《Practice Exploration of “Inter...》评分：0.0
文献《Practice Exploration of “Internet plus Deep Learning” Mixed Teaching System of Medical Physics》未通过Qwen过滤。
文献《基于深度学习的工业机器人运动控制算法优化研究...》评分：0.0
文献《基于深度学习的工业机器人运动控制算法优化研究》未通过Qwen过滤。
文献《MicroDIG: 基于自监督学习的大规模微生物组纵向数据有...》评分：0.85
文献《MicroDIG: 基于自监督学习的大规模微生物组纵向数据有向互作估计》通过Qwen过滤。
文献《基于机器学习和深度学习的抗菌肽预测研究进展...》评分：0.25
文献《基于机器学习和深度学习的抗菌肽预测研究进展》未通过Qwen过滤。
文献《基于ANSYS的土与深基坑地连墙相互作用研究...》评分：0.0
文献《基于ANSYS的土与深基坑地连墙相互作用研究》未通过Qwen过滤。
文

## 第二步：将检索到的文献上传到Zetro进行文献管理

In [25]:
import os
import json
import yaml
import requests

# 请在此处填写你的 Zotero 用户 ID 和 API Key

USER_ID = "11510297"
API_KEY = "Gvq4FIgC0J6eggLUigAn2E5a"

# 构造用于上传文献的 Zotero API URL
# 新建项时应 POST 到 /users/{user_id}/items 接口
ZOTERO_UPLOAD_URL = f"https://api.zotero.org/users/{USER_ID}/items"

# ---------------------------
# 函数：load_filtered_items
# 作用：从 YAML 文件中读取第一步筛选好的文献信息
# ---------------------------
def load_filtered_items(filename="filtered_items.yaml"):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            items = yaml.safe_load(f)
        print(f"成功读取 {len(items)} 篇筛选后的文献。")
        return items
    except Exception as e:
        print(f"读取文件 {filename} 时出错：", e)
        return []

# ---------------------------
# 函数：convert_to_zotero_format
# 作用：转换单个文献的信息为符合 Zotero API 要求的数据格式
#
# 示例中假定每篇文献包含 "title", "abstract", "authors", "date", "DOI" 字段，
# 转换为 Zotero 项（如期刊论文）的格式。你可能需要补充更多必需字段。
# ---------------------------
def convert_to_zotero_format(item):
    # 默认转换为期刊论文；实际上传时请确保数据包含 Zotero 要求的所有必要字段。
    zotero_item = {
        "itemType": "journalArticle",
        "title": item.get("title", "无标题"),
        "abstractNote": item.get("abstract", ""),
        "creators": [],
        "date": str(item.get("date", "")),
        "DOI": item.get("DOI", "")
    }
    # 假设 "authors" 字段为一个列表，每个作者为字典，包含 "name" 或 "firstName"/"lastName" 信息
    for author in item.get("authors", []):
        # 如果有单独的 firstName 与 lastName
        if "firstName" in author and "lastName" in author:
            creator = {
                "creatorType": "author",
                "firstName": author["firstName"],
                "lastName": author["lastName"]
            }
        else:
            # 若只有一个姓名字段，
            creator = {"creatorType": "author", "lastName": author.get("name", "").strip()}
        zotero_item["creators"].append(creator)
    
    return zotero_item

# ---------------------------
# 函数：upload_items_to_zotero
# 作用：将转换后的文献数据上传到 Zotero
#
# 上传时，将文献数据以 JSON 数组传递给接口。请求头中通过 "Zotero-API-Key" 传递 API Key。
# ---------------------------
def upload_items_to_zotero(items):
    headers = {
        "Zotero-API-Key": API_KEY,
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    # 将每篇文献转换为 Zotero 格式
    zotero_items = [convert_to_zotero_format(item) for item in items]
    
    # 这里假定接口支持批量上传文档。根据 Zotero API，不同接口可能要求的请求体结构略有不同，
    # 有时需要逐条 POST。请根据实际文档修改。
    payload = json.dumps(zotero_items, ensure_ascii=False)
    
    try:
        response = requests.post(ZOTERO_UPLOAD_URL, headers=headers, data=payload)
        if response.status_code in (200, 201):
            print("文献上传成功！")
            try:
                resp_json = response.json()
                print("服务器返回的信息：")
                print(json.dumps(resp_json, ensure_ascii=False, indent=2))
            except Exception as err:
                print("解析服务器返回数据时出错：", err)
        else:
            print(f"上传失败，状态码: {response.status_code}")
            print("响应内容：", response.text)
    except Exception as e:
        print("上传过程中出现错误：", e)

# ---------------------------
# 主函数
# ---------------------------
def main():
    items = load_filtered_items("filtered_items.yaml")
    if items:
        print("开始上传文献到 Zotero ...")
        upload_items_to_zotero(items)
    else:
        print("没有读取到可上传的文献。")

if __name__ == "__main__":
    main()


成功读取 2 篇筛选后的文献。
开始上传文献到 Zotero ...
文献上传成功！
服务器返回的信息：
{
  "successful": {
    "0": {
      "key": "I2QNRAH8",
      "version": 1092,
      "library": {
        "type": "user",
        "id": 11510297,
        "name": "ZhangRui218",
        "links": {
          "alternate": {
            "href": "https://www.zotero.org/zhangrui218",
            "type": "text/html"
          }
        }
      },
      "links": {
        "self": {
          "href": "https://api.zotero.org/users/11510297/items/I2QNRAH8",
          "type": "application/json"
        },
        "alternate": {
          "href": "https://www.zotero.org/zhangrui218/items/I2QNRAH8",
          "type": "text/html"
        }
      },
      "meta": {
        "parsedDate": "2025",
        "numChildren": 0
      },
      "data": {
        "key": "I2QNRAH8",
        "version": 1092,
        "itemType": "journalArticle",
        "title": "MicroDIG: 基于自监督学习的大规模微生物组纵向数据有向互作估计",
        "creators": [],
        "abstractNote": "",
        "pu